In [1]:
import numpy as np
import pandas as pd
import sys, time, copy, time, random
import pickle as pkl
from sklearn import covariance
from sklearn.covariance import GraphicalLasso
import pandas as pd 
import numpy.linalg as LA
import networkx as nx
import cvxpy as cp
from numpy.linalg import norm as nm
from numpy.linalg import inv, pinv 
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')
from control import dare
import csv
import os
from sklearn.metrics import roc_auc_score

/Users/chyj528/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def action_series(noise_level, graph, flag_m, T):
    n = graph.shape[0]
    G = graph
    spectral_radius = np.max([np.abs(i) for i in np.linalg.eigvals(G)])
    G = G.T/(spectral_radius+0.01)
    # generate alpha
    alpha_cov = np.identity(n)
    alpha_com = 10*np.random.multivariate_normal(mean = np.zeros(n), cov = alpha_cov)

    # generate u n*T
    u_cov = np.identity(n) # Sw
    U_com = np.random.multivariate_normal(mean = np.zeros(n), cov = u_cov, size = T).reshape(T, n)
    
    # generate x0
    x0_cov = np.identity(n) # Sw
    x0_com = 10*np.random.multivariate_normal(mean = np.zeros(n), cov = x0_cov)
    # generate x series n*(T+1)
    X_com = np.zeros((T+1,n))
    X_com[0] = x0_com
    
    noise = noise_level * np.random.multivariate_normal(mean = np.zeros(n), cov = np.identity(n), size = T).reshape(T, n)
    for i in range(T):
        X_com[i+1] = alpha_com + X_com[i]@G.T + U_com[i]*flag_m + noise[i]
    X_com = X_com.T
    U_com = U_com.T
    return X_com, U_com, alpha_com, G, u_cov

def Y_Z_V_series(X_com, U_com):
    n, m = X_com.shape
    X_com = X_com.T
    U_com = U_com.T
    Y_com = np.zeros((m-2, n))
    Z_com = np.zeros((m-2, n))
    V_com = np.zeros((m-2, n))
    #noise = noise_level * np.random.multivariate_normal(mean = np.zeros(n), cov = np.identity(n), size = m-1).reshape(m-1, n)
    for i in range(m-2):
        Y_com[i] = X_com[i+1] - X_com[i] #+ noise[i]
        Z_com[i] = X_com[i+2] - X_com[i+1] #+ noise[i+1]
        V_com[i] = U_com[i+1] - U_com[i]
    Y_com = Y_com.T
    Z_com = Z_com.T
    V_com = V_com.T
    return Y_com, Z_com, V_com

## Algorithm 1: SSSI

In [3]:
def learning_graph_1(theta, Y_com, Z_com, V_com, B, G_truth, u_cov):
    G = cp.Variable((n,n),name = 'G')
    obj = cp.Minimize(0.5*cp.norm(Z_com - G@Y_com - B*V_com,'fro')**2 + theta*cp.pnorm(G, 1))
    prob = cp.Problem(obj)
    prob.solve()
    G_learn_1 = G.value
    K, G_stable = project2stable(G_learn_1, u_cov)
    relative_error = np.linalg.norm(G_stable - G_truth,'fro')/np.linalg.norm(G_truth,'fro')
    return G_stable

def project2stable(G_learn, u_cov):
    delta = 0.00000001
    Q = np.identity(n)
    X, L, K = dare(G_learn, np.identity(n), Q, inv(2*delta*u_cov)) 
    return K, G_learn - K

## Noise Level


In [ ]:
folder_path = './village_adj'
T = 40 
noise_levels = [0.4,0.5,0.6,0.7,0.8]
df_graph = []
df_error = []
df_noise = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        adjacency_matrix = np.loadtxt(file_path, delimiter=',')
        n = adjacency_matrix.shape[0]
        flag_m = np.random.choice([0,1],size = n)
        B = flag_m.reshape(n,1) @ np.ones((1,T-1))
        
        for noise_level in noise_levels:
            X_com, U_com, alpha_com, G_truth, u_cov = action_series(noise_level, adjacency_matrix, flag_m, T)
            Y_com, Z_com, V_com = Y_Z_V_series(X_com, U_com)
            learn1_theta = 0
            G_learn = learning_graph_1(learn1_theta, Y_com, Z_com, V_com, B, G_truth, u_cov)
            A_flat = adjacency_matrix.flatten()
            G_learn_flat = G_learn.flatten()
            auc_roc = roc_auc_score(A_flat, G_learn_flat)
            
            df_graph.append(int(filename[0]))
            df_noise.append(noise_level)
            df_error.append(auc_roc)
            print('Graph ',int(filename[0]), ', Noise level ', noise_level,' auc_roc: ',auc_roc)
columns = {'graph':df_graph, 'noise level':df_noise, 'AUC':df_error}
df_performance = pd.DataFrame(columns)
df_performance.to_csv('./Indian_noise_performance.csv')


Graph  2 , Noise level  0.4  auc_roc:  0.9683987431866236
Graph  2 , Noise level  0.5  auc_roc:  0.44001458030176765
Graph  2 , Noise level  0.6  auc_roc:  0.959144857385074


## T

In [4]:
folder_path = './village_adj'

noise_level = 0

T_range = [5,10,15,20,25,30,35,40]

df_graph = []
df_error = []
df_T = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        adjacency_matrix = np.loadtxt(file_path, delimiter=',')
        n = adjacency_matrix.shape[0]
        flag_m = np.random.choice([0,1],size = n)
        for T in T_range:
            B = flag_m.reshape(n,1) @ np.ones((1,T-1))
            X_com, U_com, alpha_com, G_truth, u_cov = action_series(noise_level, adjacency_matrix, flag_m, T)
            Y_com, Z_com, V_com = Y_Z_V_series(X_com, U_com)
            learn1_theta = 0
            G_learn = learning_graph_1(learn1_theta, Y_com, Z_com, V_com, B, G_truth, u_cov)
            A_flat = adjacency_matrix.flatten()
            G_learn_flat = G_learn.flatten()
            auc_roc = roc_auc_score(A_flat, G_learn_flat)
            
            df_graph.append(int(filename[0]))
            df_T.append(T)
            df_error.append(auc_roc)
            print('Graph ',int(filename[0]), ', T ', T,' auc_roc: ',auc_roc)
columns = {'graph':df_graph, 'T':df_T, 'AUC':df_error}
df_performance = pd.DataFrame(columns)
df_performance.to_csv('./Indian_T_performance.csv')


Graph  7 , T  5  auc_roc:  0.8522722847693105
Graph  7 , T  10  auc_roc:  0.9179078240965388
Graph  7 , T  15  auc_roc:  0.9276878616432646
Graph  7 , T  20  auc_roc:  0.9605122814531042
Graph  7 , T  25  auc_roc:  0.9596277381145816
Graph  7 , T  30  auc_roc:  0.963213164452131
Graph  7 , T  35  auc_roc:  0.9643569157124261
Graph  7 , T  40  auc_roc:  0.9690192593407031
Graph  3 , T  5  auc_roc:  0.8817540629198464
Graph  3 , T  10  auc_roc:  0.9336976492010557
Graph  3 , T  15  auc_roc:  0.9553398667403209
Graph  3 , T  20  auc_roc:  0.9609597561225118
Graph  3 , T  25  auc_roc:  0.9639198395973566
Graph  3 , T  30  auc_roc:  0.9671958862723435
Graph  3 , T  35  auc_roc:  0.9730598577381318
Graph  3 , T  40  auc_roc:  0.9799670601714507
Graph  4 , T  5  auc_roc:  0.8604897506309551
Graph  4 , T  10  auc_roc:  0.9140334386166301
Graph  4 , T  15  auc_roc:  0.9253808929719105
Graph  4 , T  20  auc_roc:  0.9438840171019482
Graph  4 , T  25  auc_roc:  0.9452375060575127
Graph  4 , T  30 

Graph  1 , T  5  auc_roc:  0.8468453529811191
Graph  1 , T  10  auc_roc:  0.9075955624456882
Graph  1 , T  15  auc_roc:  0.943823235618334
Graph  1 , T  20  auc_roc:  0.9473993456334825
Graph  1 , T  25  auc_roc:  0.957799469072502
Graph  1 , T  30  auc_roc:  0.956539648381404
Graph  1 , T  35  auc_roc:  0.9612105872490018
Graph  1 , T  40  auc_roc:  0.9701555960360044
Graph  7 , T  5  auc_roc:  0.8771163361607826
Graph  7 , T  10  auc_roc:  0.9402177422436192
Graph  7 , T  15  auc_roc:  0.9475294207550053
Graph  7 , T  20  auc_roc:  0.9622656847012073
Graph  7 , T  25  auc_roc:  0.9596435599113557
Graph  7 , T  30  auc_roc:  0.9621609449029498
Graph  7 , T  35  auc_roc:  0.9618602896224973
Graph  7 , T  40  auc_roc:  0.9723496962402569
Graph  1 , T  5  auc_roc:  0.8750738811439558
Graph  1 , T  10  auc_roc:  0.9337847635409361
Graph  1 , T  15  auc_roc:  0.9638777027364261
Graph  1 , T  20  auc_roc:  0.9633411614509861
Graph  1 , T  25  auc_roc:  0.974072643881065
Graph  1 , T  30  au

Graph  6 , T  5  auc_roc:  0.9081138842709671
Graph  6 , T  10  auc_roc:  0.9472848424461328
Graph  6 , T  15  auc_roc:  0.967302856854049
Graph  6 , T  20  auc_roc:  0.9798718774875859
Graph  6 , T  25  auc_roc:  0.9848091429437854
Graph  6 , T  30  auc_roc:  0.9843921761874076
Graph  6 , T  35  auc_roc:  0.9930812399255597
Graph  6 , T  40  auc_roc:  0.9947346665439233
Graph  2 , T  5  auc_roc:  0.8853729157156919
Graph  2 , T  10  auc_roc:  0.9344561678394406
Graph  2 , T  15  auc_roc:  0.9556643318707463
Graph  2 , T  20  auc_roc:  0.9471764233832777
Graph  2 , T  25  auc_roc:  0.9640050123500472
Graph  2 , T  30  auc_roc:  0.9596853059361354
Graph  2 , T  35  auc_roc:  0.974331010592597
Graph  2 , T  40  auc_roc:  0.9807169185385598
Graph  5 , T  5  auc_roc:  0.8187241760058707
Graph  5 , T  10  auc_roc:  0.8933862245361361
Graph  5 , T  15  auc_roc:  0.9181187967686323
Graph  5 , T  20  auc_roc:  0.9229767681275104
Graph  5 , T  25  auc_roc:  0.9466279196941495
Graph  5 , T  30  

Graph  5 , T  5  auc_roc:  0.8918270334702463
Graph  5 , T  10  auc_roc:  0.9209310030375196
Graph  5 , T  15  auc_roc:  0.9429575620309546
Graph  5 , T  20  auc_roc:  0.9550701697811462
Graph  5 , T  25  auc_roc:  0.9598326008263225
Graph  5 , T  30  auc_roc:  0.9722821920808498
Graph  5 , T  35  auc_roc:  0.9675947023316583
Graph  5 , T  40  auc_roc:  0.9679937584245876
Graph  5 , T  5  auc_roc:  0.8486969270640075
Graph  5 , T  10  auc_roc:  0.909478055360758
Graph  5 , T  15  auc_roc:  0.9255577656757044
Graph  5 , T  20  auc_roc:  0.9592264695343773
Graph  5 , T  25  auc_roc:  0.9593401618225064
Graph  5 , T  30  auc_roc:  0.9628866370574293
Graph  5 , T  35  auc_roc:  0.9778016169505499
Graph  5 , T  40  auc_roc:  0.9749744445805856
Graph  6 , T  5  auc_roc:  0.8759077780931234
Graph  6 , T  10  auc_roc:  0.9493683239335233
Graph  6 , T  15  auc_roc:  0.9406923222719028
Graph  6 , T  20  auc_roc:  0.957160116588414
Graph  6 , T  25  auc_roc:  0.9741448349820238
Graph  6 , T  30  